<a href="https://colab.research.google.com/github/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/blob/main/Data_exploration_Curitiba_rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages 

In [1]:
%%capture
!pip install geopandas
!pip install outlier-removal-101703289
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from outlier_removal.outlier import remove_outlier
!pip install unidecode
import unidecode

In [2]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly as py
#import plotly.figure_factory as ff
from plotly import tools
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

# Import data 

Import extracted data 

In [48]:
url = 'https://raw.githubusercontent.com/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/main/data_mercado_livre_ap_curitiba_rent.csv'
da_ml = pd.read_csv(url)
#da_ml = pd.read_csv('dados_mercado_livre_curitiba.csv')

Import geodata

In [4]:
!wget 'http://ippuc.org.br/geodownloads/SHAPES/DIVISA_DE_BAIRROS.zip'
!unzip '/content/DIVISA_DE_BAIRROS.zip'

--2021-02-27 11:39:58--  http://ippuc.org.br/geodownloads/SHAPES/DIVISA_DE_BAIRROS.zip
Resolving ippuc.org.br (ippuc.org.br)... 201.11.35.17
Connecting to ippuc.org.br (ippuc.org.br)|201.11.35.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337191 (329K) [application/x-zip-compressed]
Saving to: ‘DIVISA_DE_BAIRROS.zip’

DIVISA_DE_BAIRROS.z 100%[===================>] 329.29K   364KB/s    in 0.9s    

2021-02-27 11:39:59 (364 KB/s) - ‘DIVISA_DE_BAIRROS.zip’ saved [337191/337191]

Archive:  /content/DIVISA_DE_BAIRROS.zip
  inflating: DIVISA_DE_BAIRROS.shx   
  inflating: DIVISA_DE_BAIRROS.dbf   
  inflating: DIVISA_DE_BAIRROS.prj   
  inflating: DIVISA_DE_BAIRROS.sbn   
  inflating: DIVISA_DE_BAIRROS.sbx   
  inflating: DIVISA_DE_BAIRROS.shp   


Import IDHM data 

In [47]:
url = 'https://raw.githubusercontent.com/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/main/IDMH_Bairros_Curitiba.xlsx'
idhm = pd.read_excel(url)
idhm = idhm[['NOME_UDH', 'IDHM']]
idhm = idhm.rename(columns={"NOME_UDH": "bairro"})
idhm['bairro'] = idhm['bairro'].str.capitalize().apply(unidecode.unidecode)
idhm.head()

,bairro,IDHM
0,Abranches,0.800
1,Agua verde,0.946
2,Ahu,0.929
3,Alto boqueirao,0.750
4,Alto da gloria,0.949


# Filtering data 

In [49]:
da_ml = remove_outlier(da_ml)

In [50]:
pd.options.display.float_format = "{:.2f}".format

In [51]:
da_ml.replace(',','.', regex=True,inplace=True)
da_ml['areas'] = da_ml['areas'].astype(float)

In [52]:
da_ml

,bairro,precos,quartos,areas
0,Abranches,2500.00,2.00,80.00
1,Abranches,2100.00,2.00,65.00
2,Abranches,2500.00,2.00,80.00
3,Abranches,2100.00,2.00,65.00
4,Abranches,600.00,1.00,35.00
...,...,...,...,...
911,Agua-Verde,1500.00,1.00,46.00
912,Agua-Verde,1200.00,1.00,29.00
913,Agua-Verde,1230.00,1.00,31.00
914,Agua-Verde,1500.00,1.00,30.00


In [53]:
da_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811 entries, 0 to 915
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bairro   802 non-null    object 
 1   precos   811 non-null    float64
 2   quartos  789 non-null    float64
 3   areas    802 non-null    float64
dtypes: float64(3), object(1)
memory usage: 31.7+ KB


In [54]:
da_ml['bairro'] = da_ml['bairro'].astype(str)

In [55]:
da_ml['bairro'] = da_ml['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

In [56]:
da_ml[da_ml['bairro']=='Agua verde']

,bairro,precos,quartos,areas
869,Agua verde,2200.00,1.00,32.00
870,Agua verde,1474.00,2.00,76.00
871,Agua verde,1990.00,4.00,270.00
873,Agua verde,1100.00,2.00,55.00
874,Agua verde,1690.00,3.00,72.00
875,Agua verde,1100.00,2.00,55.00
876,Agua verde,990.00,2.00,65.00
877,Agua verde,1500.00,2.00,81.00
878,Agua verde,1400.00,1.00,49.00
880,Agua verde,2330.00,3.00,151.00


merged idhm with extracted data from mercado livre 

In [59]:
da_ml = da_ml.set_index('bairro').join(idhm.set_index('bairro'))
da_ml = da_ml_teste.reset_index()
da_ml = da_ml.drop(['index'], axis=1)

In [60]:
da_ml.head(20)

,bairro,precos,quartos,areas,IDHM
0,Abranches,2500.00,2.00,80.00,0.80
1,Abranches,2100.00,2.00,65.00,0.80
2,Abranches,2500.00,2.00,80.00,0.80
3,Abranches,2100.00,2.00,65.00,0.80
4,Abranches,600.00,1.00,35.00,0.80
5,Abranches,855.00,1.00,28.00,0.80
6,Abranches,855.00,1.00,28.00,0.80
7,Abranches,855.00,1.00,28.00,0.80
8,Agua verde,2200.00,1.00,32.00,0.95
9,Agua verde,1474.00,2.00,76.00,0.95


# Data exploration

In [61]:
da_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
precos,811.00,1303.43,615.21,2.00,850.00,1170.00,1600.00,3250.00
quartos,789.00,1.89,0.84,1.00,1.00,2.00,3.00,5.00
areas,802.00,64.75,49.11,17.00,37.00,53.00,74.00,801.00
IDHM,632.00,0.87,0.08,0.70,0.80,0.90,0.93,0.95


In [35]:
indexNames = da_ml[da_ml['areas'] >= 350].index

In [36]:
da_ml.drop(indexNames , inplace=True)

In [37]:
da_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
precos,758.00,1296.74,607.63,2.00,850.00,1160.00,1600.00,3250.00
quartos,737.00,1.86,0.83,1.00,1.00,2.00,3.00,5.00
areas,749.00,62.21,36.90,17.00,36.00,53.00,72.00,300.00
IDHM,579.00,0.87,0.08,0.70,0.80,0.90,0.93,0.95


In [38]:
da_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 758 entries, Abranches to Xaxim
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   precos   758 non-null    float64
 1   quartos  737 non-null    float64
 2   areas    749 non-null    float64
 3   IDHM     579 non-null    float64
dtypes: float64(4)
memory usage: 29.6+ KB


In [39]:
da_ml.isnull().sum()

precos       0
quartos     21
areas        9
IDHM       179
dtype: int64

In [45]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [46]:
print_full(da_ml)

                                  precos              quartos                areas                 IDHM
bairro                                                                                                 
Abranches                       2,500.00                 2.00                80.00                 0.80
Abranches                       2,100.00                 2.00                65.00                 0.80
Abranches                       2,500.00                 2.00                80.00                 0.80
Abranches                       2,100.00                 2.00                65.00                 0.80
Abranches                         600.00                 1.00                35.00                 0.80
Abranches                         855.00                 1.00                28.00                 0.80
Abranches                         855.00                 1.00                28.00                 0.80
Abranches                         855.00                 1.00   

In [40]:
da_ml['bairro'].value_counts()

KeyError: ignored

In [ ]:
da_group = da_ml.groupby(['bairro']).median().sort_values('precos', ascending=False)
da_group

In [41]:
fig = make_subplots(rows=3, cols=1)
preços = go.Histogram(x=da_ml['precos'], name="preços")
quartos = go.Histogram(x=da_ml['quartos'], name="quartos")
áreas = go.Histogram(x=da_ml['areas'], name="áreas")

fig.append_trace(preços, 1, 1)
fig.append_trace(quartos, 2, 1)
fig.append_trace(áreas, 3, 1)
fig.show()

In [42]:
px.scatter(data_frame=da_ml, x="precos", y="areas")

In [43]:
px.box(data_frame=da_ml, x="quartos", y="areas")

In [ ]:
px.box(data_frame=da_ml, x="quartos", y="precos")

In [ ]:
da_price = da_group.drop('quartos', 1).drop('areas', 1).reset_index()
da_price['bairro'] = da_price['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

In [ ]:
da_areas_precos = da_group.drop('quartos', 1).reset_index()
da_areas_precos['bairro'] = da_areas_precos['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

In [ ]:
px.scatter(data_frame=da_areas_precos, x="precos", y="bairro", color='areas')

In [ ]:
da_price

In [ ]:
da_group

In [44]:
correlations = da_ml.corr()
fig = px.imshow(correlations, labels=dict(color="Correlação"), title="Correlation between numerical variables")
#fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
counts = da_ml['bairro'].value_counts()

In [ ]:
counts.values

In [ ]:
#da_ml = da_ml[da_ml['bairro'].isin(counts[counts > 3].index)]

In [ ]:
da_ml['bairro'].value_counts().values

In [ ]:
da_ml.groupby('bairro').count()

In [ ]:
#da_ml.to_csv('data_rent_ap_curitiba_mercado_livre.csv')

In [ ]:
da_group.head()

In [ ]:
#da_price = da_group.drop('quartos', 1).drop('areas', 1).reset_index()

In [ ]:
#da_price['bairro'] = da_price['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

geo files 

In [ ]:
db = gpd.read_file('DIVISA_DE_BAIRROS.shp')

In [ ]:
db.head()

In [ ]:
db['bairro'] = db['NOME'].str.capitalize().apply(unidecode.unidecode)

In [ ]:
db['bairro'].head()

In [ ]:
db['bairro'] = db['bairro'].replace('Cidade industrial de curitiba', 'Cidade industrial')

In [ ]:
merged = db.set_index('bairro').join(da_price.set_index('bairro'))
merged = merged.reset_index()
#merged = merged.fillna(0)

In [ ]:
merged['precos'] = merged['precos'].fillna(merged.groupby('NM_REGIONA')['precos'].transform('median'))

In [ ]:
merged.head()

In [ ]:
 merged.groupby(['bairro']).median().sort_values('precos', ascending=True)

In [ ]:
 merged.groupby(['NM_REGIONA']).median().sort_values('precos', ascending=False)

In [ ]:
merged.head()

In [ ]:
merged = merged.to_crs(epsg=4326) # convert the coordinate reference system to lat/long
lga_json = merged.__geo_interface__ #covert to geoJSON

In [ ]:
MAPBOX_ACCESSTOKEN = 'pk.eyJ1IjoiZm1hcmNlbG5ldmVzIiwiYSI6ImNranN3NDkzNTcxYmIzM255dzEzMzA0bm4ifQ.Exq1iIAFQfPb3_sSTMiDZw'

zmin = 450
zmax = 2300

# Set the data for the map
data = go.Choroplethmapbox(
        geojson = lga_json,             #this is your GeoJSON
        locations = merged.index,    #the index of this dataframe should align with the 'id' element in your geojson
        z = merged.precos, #sets the color value
        text = merged.bairro,    #sets text for each shape
        colorbar=dict(thickness=20, ticklen=3,outlinewidth=0), #adjusts the format of the colorbar
        marker_line_width=1, marker_opacity=0.7, colorscale="Viridis", #adjust format of the plot
        zmin=zmin, zmax=zmax,           #sets min and max of the colorbar
        hovertemplate = "<b>%{text}</b><br>" +
                    "%{z}<br>" +
                    "<extra></extra>")  # sets the format of the text shown when you hover over each shape

# Set the layout for the map
layout = go.Layout(
    title = {'text': f"Apartment Rent prices, Curitiba-PR",
            'font': {'size':24}},       #format the plot title
    mapbox1 = dict(
        domain = {'x': [0, 1],'y': [0, 1]}, 
        center = dict(lat=-25.5139519 , lon=-49.3883419),
        accesstoken = MAPBOX_ACCESSTOKEN, 
        zoom = 10),                      
    autosize=True,
    height=650,
    margin=dict(l=0, r=0, t=40, b=0))

# Generate the map
fig=go.Figure(data=data, layout=layout)
fig.show()